In [2]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn import neighbors, metrics
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from tabulate import tabulate

import time

In [3]:
bone_marrow_transplant_children = fetch_ucirepo(id=565)

In [4]:
diabetesRepo = fetch_ucirepo(id=296)

/opt/homebrew/lib/python3.11/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


# Prepare datasets

In [29]:
randomState = 22020
labelEncoder = LabelEncoder()
boneData, boneTarget = shuffle(bone_marrow_transplant_children.data.features, bone_marrow_transplant_children.data.targets, random_state=randomState)
diabetesData, diabetesTarget = shuffle(diabetesRepo.data.features, diabetesRepo.data.targets, random_state=randomState)

numericBoneData = boneData.select_dtypes(include='number')
numericBoneTarget = boneTarget.select_dtypes(include='number')

numericDiabetesData = diabetesData.select_dtypes(include='number')
numericDiabetesTarget = labelEncoder.fit_transform(diabetesTarget)

numericBoneData = numericBoneData.fillna(0)
numericBoneTarget = numericBoneTarget.fillna(0)

xBoneTrain, xBoneTest, yBoneTrain, yBoneTest = train_test_split(numericBoneData, numericBoneTarget, test_size=0.33, random_state=(randomState+1))
xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest = train_test_split(numericDiabetesData, numericDiabetesTarget, test_size=0.33, random_state=(randomState+1))

/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Helper functions

In [44]:
columns = ["Model", "Accuracy", "Precision", "Recall", "Training Time", "Testing Time"]
def printTable(cols, data):
    print(tabulate(data, headers=cols, tablefmt="grid"))

def getMeasures(classifier, X_train, X_test, y_train, y_test):
    startTimeTrain = time.time()
    classifier.fit(X_train, y_train)
    endTimeTrain = time.time()
    trainingTime = endTimeTrain - startTimeTrain

    startTimeTest = time.time()
    yTestPredicted = classifier.predict(X_test)
    endTimeTest = time.time()
    testingTime = endTimeTest - startTimeTest

    accuracy = metrics.accuracy_score(y_test, yTestPredicted)
    precision = metrics.precision_score(y_test, yTestPredicted, average="micro")
    recall = metrics.recall_score(y_test, yTestPredicted, average='macro')
    return (accuracy, precision, recall, trainingTime, testingTime)

def collectIterationData(classifierName, classifier, X_train, X_test, y_train, y_test):
    accuracy, precision, recall, trainingTime, testingTime = getMeasures(classifier, X_train, X_test, y_train, y_test)
    return [classifierName, str(accuracy), str(precision), str(recall), str(trainingTime) + "s", str(testingTime) + "s"]

## KNN

In [45]:
noNeighbors = [10, 20, 40]

def calcKNN(X_train, X_test, y_train, y_test):
    data = []
    for neighbor in noNeighbors:
        classifierName = "KNN (" + str(neighbor) + " Neighbors)"
        classifier = neighbors.KNeighborsClassifier(neighbor)

        dataIteration = collectIterationData(classifierName, classifier, X_train, X_test, y_train, y_test)
        data.append(dataIteration)
    return data


## Perceptron

In [46]:
# With the help of https://www.geeksforgeeks.org/sklearn-perceptron/, but barely.

pLearningRate = 0.1
iterations = 50

def calcPerceptron(X_train, X_test, y_train, y_test):
    data = []

    classifierName = "Perceptron"
    classifier = Perceptron(max_iter=iterations, eta0=pLearningRate, random_state=randomState)
    dataIteration = collectIterationData(classifierName, classifier, X_train, X_test, y_train, y_test)

    data.append(dataIteration)
    
    return data



## Naive Bayes (Gaussian)

In [47]:
def calcGaussianNaiveBayes(X_train, X_test, y_train, y_test):
    data = []

    classifier = GaussianNB()
    classifierName = "Naive Bayes (Gaussian)"
    dataIteration = collectIterationData(classifierName, classifier, X_train, X_test, y_train, y_test)
    
    data.append(dataIteration)

    return data

## Naive Bayes (Bernoulli)

In [48]:
def calcBernoulliNaiveBayes(X_train, X_test, y_train, y_test):
    data = []

    classifierName = "Naive Bayes (Bernoulli)"
    classifier = BernoulliNB()
    dataIteration = collectIterationData(classifierName, classifier, X_train, X_test, y_train, y_test)
    
    data.append(dataIteration)

    return data

## Bone Marrow dataset

In [49]:
boneMarrowTable = []

boneMarrowTable += calcKNN(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)
boneMarrowTable += calcPerceptron(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)
boneMarrowTable += calcGaussianNaiveBayes(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)
boneMarrowTable += calcBernoulliNaiveBayes(xBoneTrain, xBoneTest, yBoneTrain, yBoneTest)

printTable(columns, boneMarrowTable)

+-------------------------+------------+-------------+----------+------------------------+-------------------------+
| Model                   |   Accuracy |   Precision |   Recall | Training Time          | Testing Time            |
+=========================+============+=============+==========+========================+=========================+
| KNN (10 Neighbors)      |   0.903226 |    0.903226 | 0.888889 | 0.0015759468078613281s | 0.0031599998474121094s  |
+-------------------------+------------+-------------+----------+------------------------+-------------------------+
| KNN (20 Neighbors)      |   0.83871  |    0.83871  | 0.840212 | 0.0007767677307128906s | 0.0022439956665039062s  |
+-------------------------+------------+-------------+----------+------------------------+-------------------------+
| KNN (40 Neighbors)      |   0.919355 |    0.919355 | 0.920106 | 0.0006518363952636719s | 0.0018889904022216797s  |
+-------------------------+------------+-------------+----------

/opt/homebrew/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ),

## Diabetes dataset

In [50]:
diabetesTable = []

diabetesTable += calcKNN(xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest)
diabetesTable += calcPerceptron(xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest)
diabetesTable += calcGaussianNaiveBayes(xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest)
diabetesTable += calcBernoulliNaiveBayes(xDiabetesTrain, xDiabetesTest, yDiabetesTrain, yDiabetesTest)

printTable(columns, diabetesTable)

+-------------------------+------------+-------------+----------+-----------------------+------------------------+
| Model                   |   Accuracy |   Precision |   Recall | Training Time         | Testing Time           |
+=========================+============+=============+==========+=======================+========================+
| KNN (10 Neighbors)      |   0.530685 |    0.530685 | 0.382552 | 0.029421091079711914s | 2.155259132385254s     |
+-------------------------+------------+-------------+----------+-----------------------+------------------------+
| KNN (20 Neighbors)      |   0.550755 |    0.550755 | 0.382688 | 0.024168968200683594s | 2.1443889141082764s    |
+-------------------------+------------+-------------+----------+-----------------------+------------------------+
| KNN (40 Neighbors)      |   0.558824 |    0.558824 | 0.376338 | 0.023736000061035156s | 2.6109509468078613s    |
+-------------------------+------------+-------------+----------+---------------